In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load pre-trained dialogue model
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

chat_history = ""

def chatbot(user_input, sentiment):
    global chat_history
    # Add sentiment context
    sentiment_context = f"User sentiment: {sentiment}\n"
    chat_history += f"User: {user_input}\n"  # Append user input to the chat history

    # Prepare input with sentiment and chat history
    input_text = sentiment_context + chat_history
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True)

    # Generate a response
    response_ids = model.generate(inputs["input_ids"], max_length=150, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(response_ids[:, inputs["input_ids"].shape[-1]:][0], skip_special_tokens=True)

    # Update chat history with AI response
    chat_history += f"AI: {response}\n"
    return response

# Example interaction
user_input = "I feel overwhelmed with work."
sentiment = "negative"  # Output from your sentiment analysis code
print("AI Therapist:", chatbot(user_input, sentiment))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


AI Therapist: User : I feel overwhelmed with work.


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Function to generate a response
def run(user_text, chat_history_ids):
    input_ids = tokenizer.encode(user_text + tokenizer.eos_token, return_tensors="pt")
    if chat_history_ids is None:
        bot_input_ids = input_ids
    else:
        bot_input_ids = torch.cat([chat_history_ids, input_ids], dim=-1)
    chat_history_ids = model.generate(
        bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id
    )
    resp = tokenizer.decode(
        chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True
    )
    return resp, chat_history_ids

# Interactive chat
chat_history_ids = None  # Initialize chat history

print("Chatbot: Hi! I'm here to talk. Type 'exit' to end the chat.")
while True:
    user_text = input("You: ")  # Take user input
    if user_text.lower() == "exit":  # Exit condition
        print("Chatbot: Goodbye! Take care.")
        break
    response, chat_history_ids = run(user_text, chat_history_ids)  # Generate response
    print(f"Chatbot: {response}")

Chatbot: Hi! I'm here to talk. Type 'exit' to end the chat.


KeyboardInterrupt: Interrupted by user